In [ ]:
import pandas as pd
import numpy as np

bids = pd.read_csv("bidder_dataset.csv")
bids = bids.fillna(method='ffill')

bids.head()

queries = np.loadtxt("queries.txt",dtype="str", delimiter="\n")

def greedy(bids, queries):
    revenue = 0
    for query in queries:
        match_bids = bids[(bids['Keyword']==query)&(bids['Budget']>=bids['Bid Value'])]
        if match_bids.empty:
            continue
        bid_val = match_bids['Bid Value'].max()
        adv = match_bids['Advertiser'].iloc[match_bids['Bid Value'].argmax()]
        bids.loc[bids['Advertiser']==adv,'Budget'] -= bid_val
        revenue += bid_val
    return revenue

greedy(bids, queries)

def msvv(bids, queries):
    def psi(x):
        return 1-np.exp(x-1)
    
    def best_adv(bids):
        bids = bids.assign(Fraction_Spent = (bids['Initial_Budget']-bids['Budget'])/bids['Initial_Budget'])
        bids = bids.assign(Psi = psi(bids['Fraction_Spent']))
        bids = bids.assign(Prod = bids['Bid Value']*bids['Psi'])
        return bids.Advertiser.iloc[bids.Prod.argmax()]
    
    revenue = 0
    bids = bids.assign(Initial_Budget=bids['Budget'])
    for query in queries:
        match_bids = bids[(bids['Keyword']==query)&(bids['Budget']>=bids['Bid Value'])]
        if match_bids.empty:
            continue
        adv = best_adv(match_bids)
        bid_val = float(match_bids.loc[match_bids['Advertiser']==adv,'Bid Value'])
        bids.loc[bids['Advertiser']==adv,'Budget'] -= bid_val
        revenue += bid_val
    return revenue

msvv(bids, queries)

def balance(bids, queries):
    revenue = 0
    bids = bids.assign(Initial_Budget=bids['Budget'])
    for query in queries:
        match_bids = bids[(bids['Keyword']==query)&(bids['Budget']>=bids['Bid Value'])]
        if match_bids.empty:
            continue
        adv = match_bids.Advertiser.iloc[match_bids.Budget.argmax()]
        bid_val = float(match_bids.loc[match_bids['Advertiser']==adv,'Bid Value'])
        bids.loc[bids['Advertiser']==adv,'Budget'] -= bid_val
        revenue += bid_val
    return revenue

balance(bids, queries)

np.random.seed(0)

for i in range(100):
    np.random.shuffle(queries)
    greedy(bids,queries)